# 5 Noisy Data Analysis

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

from m3util.ml.rand import set_seeds
from m3util.viz.style import set_style
from m3util.viz.printing import printer
from belearn.viz.viz import Viz
from belearn.dataset.dataset import BE_Dataset
from belearn.functions.sho import SHO_nn
from belearn.nn.analysis import find_best_model
from belearn.nn.nn import create_models
from autophyslearn.spectroscopic.nn import Multiscale1DFitter, Model
from autophyslearn.postprocessing.complex import ComplexPostProcessor

    
printing = printer(basepath = './Figures/')

set_style("printing")
set_seeds(seed=42)

%matplotlib inline

## Loads Data


In [ ]:
# Specify the filename and the path to save the file
filename = "data_raw.h5"
save_path = "./Data"

data_path = save_path + "/" + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path, SHO_fit_func_LSQF=SHO_nn)

# print the contents of the file
dataset.print_be_tree()

## Collects Training Records

Training records could either be saved as a CSV file or on DataFed. We recommend using DataFed, however, this requires and account and allocation. 

In [ ]:
import pandas as pd



In [ ]:
from belearn.nn.analysis import convert_csv_df


In [ ]:
pd_df.head()

In [ ]:
# Using DataFed
DataFed = True

if DataFed:
    from datafed_torchflow.pytorch import TorchViewer

    # instantiates the TorchViewer object -- replace path with your path
    torch_viewer = TorchViewer("2024_SHO_Fitting/Training_Benchmarks_NN_SHO")

    pd_df = torch_viewer.getModelCheckpoints()
    df_new = pd_df
    
else:

    basepath = "./Trained Models/SHO Fitter/2024-09-17_12-38-10_nn_benchmarks_noise/"
    filename = "Batch_Trainging_SpeedTest.csv"
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(basepath + "/" + filename)

    
    df_new = convert_csv_df(df)



# Benchmarking on Noisy Data

To benchmark on noisy data we conducted fits using both Adam and Trust Region Optimizers. We added noise in multiples of the standard deviation of the raw data. 

Training was saved after 900 seconds.

In [ ]:
df_new.head()

In [ ]:


# # organizes the results
# results = find_best_model(basepath, filename)

from m3util.util.IO import find_files_recursive

find_files_recursive("notebooks/Trained Models/SHO Fitter/2024-09-17_12-38-10_nn_benchmarks_noise",

In [ ]:
find_min_max_by_group(df_new, 'train_loss', find='min', exclude_kwargs={'early_stopping': True}, noise_level = 0, optimizer_name = 'Adam')

In [ ]:
import pandas as pd



# Example usage
# df is your dataframe, col_name is the column for which you want to find min or max,
# kwargs are the inclusion conditions, and exclude_kwargs are the exclusion conditions.
# Example call: 
# find_min_max_by_group(df, 'train_loss', find='min', exclude_kwargs={'noise_level': 0.2}, optimizer_name='SGD', noise_level=0.1)


## Instantiate the Visualizer

In [ ]:
# insatiate the visualization object
image_scalebar = [2000, 500, "nm", "br"]

BE_viz = Viz(dataset, printing, verbose=True, 
             SHO_ranges = [(0,1.5e-4), (1.31e6, 1.33e6), (-300, 0), (-np.pi, np.pi)],
             image_scalebar=image_scalebar)

# extracts the x and y data based on the noise
X_data_no_noise, Y_data = dataset.NN_data()

## Noise Level 0, ADAM Optimizer

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 0
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_6_1_Scaled Raw Data_noise_{noise}_optimizer_{optimizer}")

**Figure 5.1** Example visualization of the scaled, noisy (noise = 0) data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()
postprocessor = ComplexPostProcessor(dataset)

adam_model, trust_region_model = create_models(basepath, results, noise, dataset, postprocessor, SHO_nn)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_2_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.2** Visualization of the noisy (noise level 0) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_3_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.3** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_4_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.4** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

# adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

# trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)


In [ ]:
#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_5_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_5_Histogram_comparison_{noise}_noise",)

**Figure 5.5** Histogram of the fit results for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers. 

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_6_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.6** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = 0, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])

## Noise Level 1

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 1
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_5_7_Scaled Raw Data_noise{noise}_optimizer_{optimizer}")

**Figure 5.7** Example visualization of the scaled, noisy data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()

# searches the trained models for the best model
model_name_adam = basepath + "/" + results[(noise, "Adam")]['filename'].split("//")[-1] 
model_name_trust_region = basepath + "/" + results[(noise, "Trust Region CG")]['filename'].split("//")[-1] 

# instantiate the model
model_adam = SHO_Model(dataset, training=False)

# instantiate the model
model_trust_region = SHO_Model(dataset, training=False)

model_adam.load(
    model_name_adam
)

model_trust_region.load(
    model_name_trust_region
)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_8_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.8** Visualization of the noisy (noise level 1) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_9_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.9** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_10_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.10** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

# adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

# trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_11_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_11_Histogram_comparison_{noise}_noise",)

**Figure 5.11** Histogram of the fit results based on fits with noise 1 for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers.

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_12_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.12** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = noise, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])

## Noise Level 2

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 2
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_5_13_Scaled Raw Data_noise{noise}_optimizer_{optimizer}")

**Figure 5.13** Example visualization of the scaled, noisy data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()

# searches the trained models for the best model
model_name_adam = basepath + "/" + results[(noise, "Adam")]['filename'].split("//")[-1] 
model_name_trust_region = basepath + "/" + results[(noise, "Trust Region CG")]['filename'].split("//")[-1] 

# instantiate the model
model_adam = SHO_Model(dataset, training=False)

# instantiate the model
model_trust_region = SHO_Model(dataset, training=False)

model_adam.load(
    model_name_adam
)

model_trust_region.load(
    model_name_trust_region
)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_14_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.14** Visualization of the noisy (noise level 2) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_15_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.15** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_16_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.16** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_16_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_16_Histogram_comparison_{noise}_noise",)

**Figure 5.16** Histogram of the fit results based on fits with noise 1 for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers.

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_17_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.17** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = noise, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])

## Noise Level 3

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 3
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_5_18_Scaled Raw Data_noise{noise}_optimizer_{optimizer}")

**Figure 5.18** Example visualization of the scaled, noisy data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()

# searches the trained models for the best model
model_name_adam = basepath + "/" + results[(noise, "Adam")]['filename'].split("//")[-1] 
model_name_trust_region = basepath + "/" + results[(noise, "Trust Region CG")]['filename'].split("//")[-1] 

# instantiate the model
model_adam = SHO_Model(dataset, training=False)

# instantiate the model
model_trust_region = SHO_Model(dataset, training=False)

model_adam.load(
    model_name_adam
)

model_trust_region.load(
    model_name_trust_region
)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_19_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.19** Visualization of the noisy (noise level 1) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_20_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.20** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_21_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.21** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_22_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_22_Histogram_comparison_{noise}_noise",)

**Figure 5.22** Histogram of the fit results based on fits with noise 1 for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers.

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_23_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.23** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = noise, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])

## Noise Level 4

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 4
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_5_24_Scaled Raw Data_noise{noise}_optimizer_{optimizer}")

**Figure 5.24** Example visualization of the scaled, noisy data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()

# searches the trained models for the best model
model_name_adam = basepath + "/" + results[(noise, "Adam")]['filename'].split("//")[-1] 
model_name_trust_region = basepath + "/" + results[(noise, "Trust Region CG")]['filename'].split("//")[-1] 

# instantiate the model
model_adam = SHO_Model(dataset, training=False)

# instantiate the model
model_trust_region = SHO_Model(dataset, training=False)

model_adam.load(
    model_name_adam
)

model_trust_region.load(
    model_name_trust_region
)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_25_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.25** Visualization of the noisy (noise level 1) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_26_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.26** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_27_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.27** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

# adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

# trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_28_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_28_Histogram_comparison_{noise}_noise",)

**Figure 5.28** Histogram of the fit results based on fits with noise 1 for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers.

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_29_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.29** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = noise, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])

## Noise Level 5

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 5
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_5_30_Scaled Raw Data_noise{noise}_optimizer_{optimizer}")

**Figure 5.30** Example visualization of the scaled, noisy data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()

# searches the trained models for the best model
model_name_adam = basepath + "/" + results[(noise, "Adam")]['filename'].split("//")[-1] 
model_name_trust_region = basepath + "/" + results[(noise, "Trust Region CG")]['filename'].split("//")[-1] 

# instantiate the model
model_adam = SHO_Model(dataset, training=False)

# instantiate the model
model_trust_region = SHO_Model(dataset, training=False)

model_adam.load(
    model_name_adam
)

model_trust_region.load(
    model_name_trust_region
)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_31_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.31** Visualization of the noisy (noise level 1) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_32_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.32** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_33_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.33** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

# adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

# trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_34_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_34_Histogram_comparison_{noise}_noise",)

**Figure 5.34** Histogram of the fit results based on fits with noise 1 for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers.

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_35_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.35** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = noise, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])

## Noise Level 6

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 6
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_5_36_Scaled Raw Data_noise{noise}_optimizer_{optimizer}")

**Figure 5.36** Example visualization of the scaled, noisy data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()

# searches the trained models for the best model
model_name_adam = basepath + "/" + results[(noise, "Adam")]['filename'].split("//")[-1] 
model_name_trust_region = basepath + "/" + results[(noise, "Trust Region CG")]['filename'].split("//")[-1] 

# instantiate the model
model_adam = SHO_Model(dataset, training=False)

# instantiate the model
model_trust_region = SHO_Model(dataset, training=False)

model_adam.load(
    model_name_adam
)

model_trust_region.load(
    model_name_trust_region
)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_37_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.37** Visualization of the noisy (noise level 1) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_38_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.38** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_39_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.39** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_40_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_40_Histogram_comparison_{noise}_noise",)

**Figure 5.40** Histogram of the fit results based on fits with noise 1 for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers.

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_41_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.41** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = noise, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])

## Noise Level 7

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 7
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_5_42_Scaled Raw Data_noise{noise}_optimizer_{optimizer}")

**Figure 5.42** Example visualization of the scaled, noisy data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()

# searches the trained models for the best model
model_name_adam = basepath + "/" + results[(noise, "Adam")]['filename'].split("//")[-1] 
model_name_trust_region = basepath + "/" + results[(noise, "Trust Region CG")]['filename'].split("//")[-1] 

# instantiate the model
model_adam = SHO_Model(dataset, training=False)

# instantiate the model
model_trust_region = SHO_Model(dataset, training=False)

model_adam.load(
    model_name_adam
)

model_trust_region.load(
    model_name_trust_region
)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_43_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.43** Visualization of the noisy (noise level 1) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_44_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.44** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_45_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.45** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

# adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

# trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_46_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_46_Histogram_comparison_{noise}_noise",)

**Figure 5.46** Histogram of the fit results based on fits with noise 1 for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers.

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_47_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.47** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = noise, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])

## Noise Level 8

### Scaling the Data

When training the neural network it is useful to scale the data. We apply a global scaler such that the spectrum have a mean of 0 and a standard deviation of 1.

#### Visualizing the Scaled Data


In [ ]:
# Sets the dataset
noise = 8
optimizer = "Adam"

state = {"fitter": "LSQF", "resampled": True, "scaled": True, "label": "Scaled", "noise": noise}
dataset.set_attributes(**state)

BE_viz.nn_checker(state, filename=f"Figure_5_48_Scaled Raw Data_noise{noise}_optimizer_{optimizer}")

**Figure 5.48** Example visualization of the scaled, noisy data which is used for training.

### Extracts the Data and Models

In [ ]:
# extracts the x and y data based on the noise
X_data, Y_data = dataset.NN_data()

# searches the trained models for the best model
model_name_adam = basepath + "/" + results[(noise, "Adam")]['filename'].split("//")[-1] 
model_name_trust_region = basepath + "/" + results[(noise, "Trust Region CG")]['filename'].split("//")[-1] 

# instantiate the model
model_adam = SHO_Model(dataset, training=False)

# instantiate the model
model_trust_region = SHO_Model(dataset, training=False)

model_adam.load(
    model_name_adam
)

model_trust_region.load(
    model_name_trust_region
)

### Evaluate the Fit Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by visualizing the best, median, and worst fits.

We will assume that the autoencoder is working correctly and thus will not consider the test train split.

Note: we are comparing the autoencoder results to the original data, not the noisy data. 


In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': True,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}


BE_viz.MSE_compare(X_data_no_noise, [model_adam, model_trust_region, LSQF_], ["Adam", "Trust Region", "LSQF"])

#### Least Squares Fit

In [ ]:

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=LSQF_,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_49_NN_validation_noise_{noise}",
    compare_state = X_data_no_noise,
)

**Figure 5.49** Visualization of the noisy (noise level 1) fit results from the least squares fitting algorithm shows the best, median, and worst fits.

#### Neural Network with Adam Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_adam,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_50_NN_validation_noise_{noise}_Adam",
    compare_state = X_data_no_noise,
)

**Figure 5.50** Visualization of the noisy (noise level 0) fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

##### Neural Network with Trust Region Conjugate Gradient Optimizer

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_data,
    prediction=model_trust_region,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename=f"Figure_5_51_NN_validation_noise_{noise}_Trust_Region",
    compare_state = X_data_no_noise,
)

**Figure 5.51** Visualization of the noisy fit results from the neural network trained with the Adam optimizers. We shows the best, median, and worst fits.

### Histogram of Fit Results

It is useful to view the histogram of the fitting results to apply any necessary phase shifts, and to see if the results are reasonable.


In [ ]:
# make all the histograms the same bin range.

LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

dataset.set_attributes(**LSQF_)

LSQF_params = dataset.SHO_fit_results(state = LSQF_)

# adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

# trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

#BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], SHO_ranges = BE_viz.SHO_ranges, filename=f"Figure_5_52_Histogram_comparison_{noise}_noise",)
BE_viz.SHO_hist([LSQF_params, adam_params, trust_region_params], filename=f"Figure_5_52_Histogram_comparison_{noise}_noise",)

**Figure 5.52** Histogram of the fit results based on fits with noise 1 for the a-d. LSQF, e-h. neural network with ADAM, i-l. neural network with trust region optimizers.

In [ ]:
LSQF_ = {'resampled': True,
                'raw_format': 'complex',
                'fitter': 'LSQF',
                'scaled': False,
                'output_shape': 'index',
                'measurement_state': 'all',
                'resampled_bins': 165,
                'LSQF_phase_shift': 1.5707963267948966,
                'NN_phase_shift': None,
                'noise': noise}

LSQF_Params = dataset.SHO_fit_results(state = LSQF_)

adam_params = dataset.SHO_fit_results(model = model_adam, phase_shift = np.pi / 2)

trust_region_params = dataset.SHO_fit_results(model = model_trust_region, phase_shift = np.pi / 2)

BE_viz.SHO_switching_maps_test([LSQF_Params, adam_params, trust_region_params], filename=f"Figure_5_53_switching_maps_comparison_{noise}_noise", labels=["LSQF", "Adam", "SGD"])

**Figure 5.53** Visualize the switching based on the different models on the noisy data. 

In [ ]:
BE_viz.SHO_fit_movie_images(noise = noise, 
                            models = [None, model_adam, model_trust_region],
                            scalebar_= True, 
                            basepath = "Movies/SHO_NN_compare",  
                            filename="SHO_NN_compare",
                            phase_shift = [None, np.pi / 2, np.pi / 2])